In [ ]:
!pip install tsplib95

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 47.9 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.0
    Uninstalling networkx-3.0:
      Successfully uninstalled networkx-3.0


In [ ]:
import tsplib95 as tsp
import networkx as nx
import numpy as np
import random
import sys
import itertools
import bisect

In [ ]:
problem = tsp.load('/content/berlin52.txt')
print(problem.name)
G = problem.get_graph()
print(G)

berlin52
Graph named 'berlin52' with 52 nodes and 1378 edges


In [ ]:
class Solution:
    def __init__(self, graph, start, ant=None):
        self.graph = graph
        self.start = start
        self.ant = ant
        self.current = start
        self.cost = 0
        self.path = []
        self.nodes = [start]
        self.visited = set(self.nodes)

    def __iter__(self):
        return iter(self.path)

    def __eq__(self, other):
        return self.cost == other.cost

    def __lt__(self, other):
        return self.cost < other.cost

    def __contains__(self, node):
        return node in self.visited or node == self.current

    def add_node(self, node):
        self.nodes.append(node)
        self.visited.add(node)
        self._add_node(node)

    def close(self):
        self._add_node(self.start)

    def _add_node(self, node):
        edge = self.current, node
        data = self.graph.edges[edge]
        self.path.append(edge)
        self.cost += data['weight']
        self.current = node


class Solver:
    def __init__(self, alpha=1, beta=3,rho=.03, q=1):
        self.rho = rho
        self.q = q
        self.alpha=alpha
        self.beta=beta

    def solve_tsp(self, graph, iterations, gen_size):
      colony = [Ant(self.alpha, self.beta) for _ in range(gen_size)]
      best_cost = 99999999
      for u, v in graph.edges:
        graph.edges[u, v].setdefault('pheromone', 0.0001)

      for i in range(iterations):
        solutions = self.find_solutions(graph, colony)
        self.global_update(graph, solutions)
        possible_solution = sorted(solutions, key=lambda x: x.cost)[0]
        if best_cost > possible_solution.cost:
          best_cost = possible_solution.cost
          best_solution = possible_solution
      return best_solution
      

    def find_solutions(self, graph, ants):
        return [ant.tour(graph) for ant in ants]

    def global_update(self, graph, solutions):
        for edge in graph.edges:
            amount = 0
            for solution in solutions:
                if edge in solution.path:
                    amount += self.q / solution.cost
            p = graph.edges[edge]['pheromone']
            graph.edges[edge]['pheromone'] = (1 - self.rho) * p + amount

In [ ]:
class Ant:
    def __init__(self, alpha=1, beta=3):
        self.alpha = alpha
        self.beta = beta

    def tour(self, graph):
        solution = self.initialize_solution(graph)
        unvisited = self.get_unvisited_nodes(graph, solution)
        while unvisited:
            node = self.choose_destination(graph, solution.current, unvisited)
            solution.add_node(node)
            unvisited.remove(node)
        solution.close()
        return solution

    def initialize_solution(self, graph):
        start = self.get_starting_node(graph)
        return Solution(graph, start, ant=self)

    def get_starting_node(self, graph):
        return random.choice(list(graph.nodes))

    def get_unvisited_nodes(self, graph, solution):
        nodes = []
        for node in graph[solution.current]:
            if node not in solution:
                nodes.append(node)
        return nodes

    def choose_destination(self, graph, current, unvisited):
        if len(unvisited) == 1:
            return unvisited[0]
        scores = self.get_scores(graph, current, unvisited)
        return self.choose_node(unvisited, scores)

    def get_scores(self, graph, current, destinations):
        scores = []
        for node in destinations:
            edge = graph.edges[current, node]
            score = self.score_edge(edge)
            scores.append(score)
        return scores

    def choose_node(self, choices, scores):
        total = sum(scores)
        cumdist = list(itertools.accumulate(scores)) + [total]
        index = bisect.bisect(cumdist, random.random() * total)
        return choices[min(index, len(choices) - 1)]

    def score_edge(self, edge):
        weight = edge.get('weight', 1)
        if weight == 0:
            return sys.float_info.max
        pre = 1 / weight
        post = edge['pheromone']
        return post ** self.alpha * pre ** self.beta

In [ ]:
solver = Solver()

In [ ]:
solution = solver.solve_tsp(G, 50, 100)

In [ ]:
print(solution.path)
print(solution.cost)

[(24, 48), (48, 38), (38, 15), (15, 5), (5, 6), (6, 4), (4, 25), (25, 12), (12, 28), (28, 27), (27, 26), (26, 47), (47, 14), (14, 13), (13, 52), (52, 11), (11, 51), (51, 33), (33, 43), (43, 10), (10, 9), (9, 8), (8, 41), (41, 19), (19, 45), (45, 32), (32, 49), (49, 1), (1, 22), (22, 31), (31, 18), (18, 3), (3, 17), (17, 21), (21, 30), (30, 2), (2, 7), (7, 42), (42, 23), (23, 20), (20, 50), (50, 16), (16, 29), (29, 46), (46, 44), (44, 34), (34, 35), (35, 36), (36, 39), (39, 40), (40, 37), (37, 24)]
7911
